In [1]:
import os, os.path 
import numpy 
from torch.utils.data import Dataset, DataLoader
import pickle
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
import matplotlib

"""Change to the data folder"""
train_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
matplotlib.rcParams['savefig.dpi'] = 300

# Train/validation test dataset loading

In [3]:
class ArgoverseDataset(Dataset):
    
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform
        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self): #len(val_dataset)
        return len(self.pkl_list)
    
    def __getitem__(self, idx): #val_dataset[0]
        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
        if self.transform:
            data = self.transform(data)
        return data

In [4]:
train_dataset = ArgoverseDataset(data_path=train_path) 
test_dataset  = ArgoverseDataset(data_path=test_path) 

In [29]:
len(train_dataset)

205942

In [45]:
in_train, out_train = [], []
for i in tqdm(range(0, 200000)):
    val = train_dataset[i]
    p_in = val['p_in'][val['car_mask'].reshape(-1) == 1]
    p_out = val['p_out'][val['car_mask'].reshape(-1) == 1]
    for c in range(len(p_in)):
        in_train.append(p_in[c].reshape(19*2))
        out_train.append(p_out[c].reshape(30*2))
in_train = np.array(in_train)
out_train = np.array(out_train)

In [46]:
in_valid, out_valid = [], []
for i in tqdm(range(200000-1, 205942)):
    val = train_dataset[i]
    p_in = val['p_in'][val['car_mask'].reshape(-1) == 1]
    p_out = val['p_out'][val['car_mask'].reshape(-1) == 1]
    for c in range(len(p_in)):
        in_valid.append(p_in[c].reshape(19*2))
        out_valid.append(p_out[c].reshape(30*2))
in_valid = np.array(in_valid)
out_valid = np.array(out_valid)

In [105]:
x_in_train = in_train[:, list(range(0, 38, 2))]
y_in_train = in_train[:, list(range(1, 39, 2))]

x_out_train = out_train[:, list(range(0, 60, 2))]
y_out_train = out_train[:, list(range(1, 61, 2))]

In [80]:
x_in_train.shape, y_in_train.shape, x_out_train.shape, y_out_train.shape

((1760155, 19), (1760155, 19), (1760155, 30), (1760155, 30))

In [81]:
x_in_valid = in_valid[:, list(range(0, 38, 2))]
y_in_valid = in_valid[:, list(range(1, 39, 2))]

x_out_valid = out_valid[:, list(range(0, 60, 2))]
y_out_train = out_valid[:, list(range(1, 61, 2))]

In [88]:
#Use that 19 time points to predict every future points: 
#obviously should be lower than the final LR
for i in range(30):
    print(i, reg.score(x_in_valid, x_out_valid[:, i]))

0 0.9999999720564747
1 0.9999998606835677
2 0.9999995591729811
3 0.9999990666000739
4 0.9999983731784919
5 0.9999975046702431
6 0.999996436367472
7 0.9999951806544309
8 0.9999937327352634
9 0.999992081261241
10 0.9999902376357248
11 0.9999882150271086
12 0.9999859938986253
13 0.9999835605996417
14 0.9999809603371024
15 0.9999781402339497
16 0.999975159806497
17 0.9999719303981158
18 0.9999685486390444
19 0.9999649747719512
20 0.9999611945165718
21 0.9999571666565411
22 0.9999529929593971
23 0.9999486070940553
24 0.9999440279500589
25 0.9999392402715116
26 0.9999343095449696
27 0.9999291747872935
28 0.999923934296924
29 0.9999185352651137


In [49]:
regs = [LinearRegression()] * 30

In [106]:
regs[0] = LinearRegression().fit(x_in_train, x_out_train[:, 0])
pred = regs[0].predict(x_in_train)
x_in_train = np.insert(x_in_train, 19 + 0, pred, axis=1)

for i in tqdm(range(1, 30)):
    regs[i] = LinearRegression().fit(x_in_train, x_out_train[:, i])
    pred = regs[i].predict(x_in_train)
    x_in_train = np.insert(x_in_train, 19 + i, pred, axis=1)

In [109]:
x_in_train.shape

(1760155, 49)

Validation set: what is the score?

In [120]:
for i in range(30):
    print(r2_score(x_in_train[:, 19 + i], x_out_train[:, i]))

0.9999999706316878
0.9999999522101889
0.9999999314364197
0.9999999059040638
0.9999998734611978
0.9999998341014829
0.9999997857053922
0.9999997270690452
0.9999996569478273
0.9999995749600455
0.9999994801562861
0.9999993694394474
0.9999992415629441
0.9999990980554518
0.9999989351657506
0.9999987506079562
0.9999985441856927
0.9999983148635783
0.9999980609190324
0.9999977815631815
0.9999974716478127
0.9999971361935339
0.9999967684914657
0.9999963682304969
0.9999959328154526
0.9999954634669268
0.9999949583667719
0.9999944146837969
0.9999938356211475
0.9999932219293578


# New: Train test split for the difference in location approach

In [4]:
train_dataset = ArgoverseDataset(data_path=train_path) 

In [11]:
#diff points

train = []
for i in tqdm(range((len(train_dataset)))):
    val = train_dataset[i]
    c_filter = val['car_mask'].reshape(-1) == 1
    p_in = val['p_in'][c_filter]
    p_out = val['p_out'][c_filter]
    for c in range(len(p_in)):
        in_out = np.diff(np.vstack((p_in[c], p_out[c])), axis=0).flatten()
        train.append(in_out)
train = np.array(train)

In [12]:
np.random.shuffle(train)

In [18]:
len(train) * 0.8 //1


1449736.0

In [22]:
train[:int(len(train) * 0.8 //1), :].shape

(1449736, 96)

In [25]:
train_ = train[:int(len(train) * 0.8 //1), :]

In [26]:
valid_ = train[int(len(train) * 0.8 //1):, :]

In [27]:
in_train, out_train = train_[:, :36], train_[:, 36:]

In [29]:
#auto regressive and teacher-forcing

#prepare a list of 60 linear regression models, half for 30 output x, half for 30 output y

num_output = 60
input_end_idx = 36

regs = [None] * num_output

#train using labels from the ground truth output, and use the GT output to train new time point
for i in tqdm(range(num_output)):
    regs[i] = LinearRegression().fit(in_train, out_train[:, i])
    
    #append GT back to the train feature
    in_train = np.insert(in_train, input_end_idx + i, out_train[:, i], axis=1)

KeyboardInterrupt: 